In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import glob
import random

- import dataset
- import phobert tokenizer and vectorizer
- create dataloader
- define encoder
- define attention
- define decoder
- define seq2seq

WIP:
+ train loop
+ validate

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# input = 'Ngày 27/3 , Cơ_quan Cảnh_sát điều_tra Công_an TP. Hưng_Yên , tỉnh Hưng_Yên cho biết , đơn_vị vừa ra quyết_định khởi_tố vụ án , khởi_tố bị_can đối_với đối_tượng Mai_Văn_Thương ( SN 1989 , trú tại đội 11 , thôn An_Chiểu 1 , xã Liên_Phương , TP. Hưng_Yên ) để điều_tra về hành_vi trộm_cắp tài_sản. Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ . Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ . Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP. Hưng_Yên tự_thú và khai nhận toàn_bộ hành_vi phạm_tội của mình , đồng_thời giao_nộp cho cơ_quan công_an 3.120.000 đ . Hiện Công_an TP. Hưng_Yên đã thu_giữ toàn_bộ 121kg thóc đối_tượng đã trộm_cắp để trao_trả cho gia_đình ông Thịnh . Được biết Thương là đối_tượng nghiện ma_tuý từ nhiều năm nay , đã có 1 tiền_án về tội Tàng_trữ trái_phép chất ma_tuý bị TAND tỉnh Hưng_Yên xử_phạt 2 năm 3 tháng tù_giam. Ra tù năm 2016, đối_tượng này tiếp_tục có hành_vi cố_ý gây thương_tích , bị Công_an TP. Hưng_Yên ra quyết_định xử_phạt 2,5 triệu đồng . Vụ án đang được Công_an TP. Hưng_Yên hoàn_thiện hồ_sơ để xử_lý Mai_Văn_Thương theo quy_định của pháp_luật .'
# target = 'Với bản_tính ham chơi , lười làm , có nhiều tiền_án tiền_sự , lại nghiện ma_tuý , Thương đã đột_nhập vào nhà chú ruột để trộm hơn 1 tạ thóc và hơn 8 triệu đồng mang đi tiêu_xài .'

In [3]:
!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Counting objects: 100% (143827/143827), done.
remote: Compressing objects: 100% (143815/143815), done.
remote: Total 143827 (delta 11), reused 143827 (delta 11), pack-reused 0
Receiving objects: 100% (143827/143827), 194.68 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (150704/150704), done.


In [4]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.0MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
     |████████████████████████████████| 2.9MB 36.4MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=97959df931d208b0ccff6502843a354d44ddcf5753c2498915559100ac8fc021
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
!wget -O phobert.tar.gz https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -zxvf phobert.tar.gz

--2020-11-20 01:52:16--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 13.226.49.2, 13.226.49.123, 13.226.49.111, ...
Connecting to public.vinai.io (public.vinai.io)|13.226.49.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘phobert.tar.gz’

phobert.tar.gz      100%[===================>] 307.47M  71.7MB/s    in 4.4s    

2020-11-20 01:52:21 (69.8 MB/s) - ‘phobert.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [3]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

config = AutoConfig.from_pretrained('/content/PhoBERT_base_transformers/config.json')
phobert = AutoModel.from_pretrained('vinai/phobert-base', config = config)
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing BertModel: ['roberta.embeddings.position_ids', 'roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediat

In [4]:
class VietDataset(Dataset):
    def __init__(self, data_path):
        self.files_path = glob.glob(f'{data_path}/*')

    def __len__(self):
        return len(self.files_path)
        
    def __getitem__(self, idx):
        file_path = self.files_path[idx]

        with open(file_path, 'r') as f:
            file_content = f.readlines()
            target = file_content[2]
            text = ' '.join(file_content[3:]).replace('\n', '')

            input_ids = torch.tensor([tokenizer.encode(text)])
            target_ids = torch.tensor([tokenizer.encode(target)])

            with torch.no_grad():
              text = phobert(input_ids)[0]
              target = phobert(target_ids)[0]
            
            text = text.squeeze()
            target = target.squeeze()

        return text, target

In [5]:
# for i in range(5):
#   text, target = train_data[i]
#   print('Van ban:', str(i+1))
#   print(text.size())
#   print(target.size())



  # print(text[0].size())
  # print(text[1].size())
  # print(target[0].size())
  # print(target[1].size())

In [6]:
def my_collate(batch):
  text = [item[0] for item in batch]
  target = [item[1] for item in batch]

  text = pad_sequence(text)
  target = pad_sequence(target)

  return text, target


In [7]:
def make_dataloader(dataset):
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn = my_collate, num_workers=0)
    return dataloader

In [8]:
train_data = VietDataset('/content/vietnews/data/train_tokenized')
valid_data = VietDataset('/content/vietnews/data/val_tokenized')
test_data = VietDataset('/content/vietnews/data/test_tokenized')

In [9]:
subtrain = torch.utils.data.Subset(train_data, [0,1,2,3,4,5,6,7])
subval = torch.utils.data.Subset(valid_data, [0,1,2,3,4,5,6,7])
subtrain_iterator = make_dataloader(subtrain)
subval_iterator = make_dataloader(subval)

In [10]:
train_iterator = make_dataloader(train_data)
val_iterator = make_dataloader(valid_data)
test_iterator = make_dataloader(test_data)

In [36]:
for i, batch in enumerate(subtrain_iterator):
  print(batch[0].size())
  print(batch[1].size())

torch.Size([845, 4, 768])
torch.Size([48, 4, 768])
torch.Size([1592, 4, 768])
torch.Size([98, 4, 768])


In [11]:
class EncoderBiGRU(nn.Module):
  def __init__(self, input_dim, output_dim, enc_hidden_dim, dec_hidden_dim, num_layers, dropout):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(input_dim, output_dim)
    self.gru = nn.GRU(input_size = output_dim,
                        hidden_size = enc_hidden_dim,
                        num_layers = num_layers,
                        dropout = dropout,
                        bidirectional = True)
    self.out = nn.Linear(enc_hidden_dim*2, dec_hidden_dim)

  def forward(self, input):
    # input: (sent len, batch size)
    print('input', input.size(), 'encoder1')
    output = self.dropout(self.linear(input))
    print('output',output.size(), 'encoder2')
    output, hidden = self.gru(output)   # output: (sent len, batch size, hidden dim*num directions)
    hidden = self.out(torch.cat((hidden[-2,::], hidden[-1,:,:]), dim=1))   # hidden: (sent len, batch size, num directions*hidden dim)
    # h_n[-2,:,:] last states of forward pass, h_n[-1,:,:] last states of backward pass
    hidden = torch.tanh(hidden)   # hidden: (batch size, dec_hidden_dim)2q  
    print('output',output.size(), 'hidden',hidden.size(), 'encoder3')

    return output, hidden

In [12]:
class Attention(nn.Module):
  def __init__(self, enc_hidden_dim, dec_hidden_dim):
    super().__init__()
    self.attn = nn.Linear(enc_hidden_dim*2 + dec_hidden_dim, dec_hidden_dim)
    self.v = nn.Linear(dec_hidden_dim, 1, bias = False)

  def forward(self, hidden, encoder_outputs):
    batch_size = encoder_outputs.shape[1]
    print('batch_size', batch_size, 'attention1')
    src_len = encoder_outputs.shape[0]
    print('src_len', src_len, 'attention2')
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
    print('hidden', hidden.size(), 'attention3')
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    print('encoder_outputs', encoder_outputs.size(), 'attention4')
    energy = torch.cat((hidden, encoder_outputs), dim=2)
    energy = torch.tanh(self.attn(energy))
    print('energy', energy.size(), 'attention5')
    attention = self.v(energy).squeeze(2)
    print('attention', attention.size(), 'attention6')

    return F.softmax(attention, dim=1)


In [13]:
class DecoderGRU(nn.Module):
  def __init__(self, output_dim, embedding_dim, enc_hidden_dim, dec_hidden_dim, dropout, attention):
      super().__init__()
      self.output_dim = output_dim
      self.attention = attention
      self.fc = nn.Linear(output_dim, embedding_dim)
      self.gru = nn.GRU(enc_hidden_dim*2 + embedding_dim, dec_hidden_dim)
      self.out = nn.Linear(enc_hidden_dim*2 + dec_hidden_dim + embedding_dim, output_dim)
      self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, enc_out):
      print('hidden', hidden.size(), 'decoder0')
      input = input.unsqueeze(0)
      print('input', input.size(), 'decoder1')
      embedded = self.fc(input)
      print('embedded', embedded.size(), 'decoder2')

      
      attn = self.attention(hidden, enc_out)
      attn = attn.unsqueeze(1)
      print('attn', attn.size(),'decoder3')
      enc_out = enc_out.permute(1, 0, 2)
      print('end_out', enc_out.size(), 'decoder4')
      weighted = torch.bmm(attn, enc_out)
      weighted = weighted.permute(1, 0, 2)
      print('weighted', weighted.size(), 'decoder5')

      
      gru_input = torch.cat((embedded, weighted), dim=2)
      output, hidden = self.gru(gru_input, hidden.unsqueeze(0))
      
      assert (output == hidden).all()

      embedded = embedded.squeeze(0)
      output = output.squeeze(0)
      weighted = weighted.squeeze(0)
      print('output', output.size(), 'weighted', weighted.size(), 'embedded', embedded.size(), 'decoder6')


      pred = torch.cat((output, weighted, embedded), dim=1)
      pred = self.out(pred)

      print('pred', pred.size(), 'hidden', hidden.size(), 'decoder7')

      return pred, hidden.squeeze(0)

In [21]:
class seq2seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
    batch_size = src.shape[1]
    print(1)
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim
    print(2)

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    encoder_outputs, hidden = self.encoder(src)
    print(3)

    input = trg[0,:]

    for t in range(1, trg_len):
      output, hidden = self.decoder(input, hidden, encoder_outputs)
      hidden = torch.unsqueeze(hidden, dim=0)
      print('output', output.size(), 'hidden', hidden.size(), 's2s1')
      outputs[t] = output
      print('outputs', outputs.size(), 's2s2')
      # teacher_force = random.random() < teacher_forcing_ratio

      row_sum_output = torch.sum(output, dim=1)
      top1 = output[row_sum_output.argmax()]
      top1 = torch.unsqueeze(top1, dim=0)

      # top1 = output.argmax(0)
      print('top1', top1.size(), 's2s3')
      input = top1
      # input = trg[t] if teacher_force else top1
      print('input', input.size(), 's2s4')
      

    return outputs

In [68]:
x = torch.rand((4,2))
torch.unsqueeze(x, dim=0).size()

torch.Size([1, 4, 2])

In [22]:
input_dim = 768
output_dim = 768
enc_hidden_dim = 2050
dec_hidden_dim = 2050
enc_dropout = 0.5
dec_dropout = 0.5
enc_layers = 2

attn = Attention(enc_hidden_dim, dec_hidden_dim)
enc = EncoderBiGRU(input_dim, output_dim, enc_hidden_dim, dec_hidden_dim, enc_layers, enc_dropout)
dec = DecoderGRU(output_dim, output_dim, enc_hidden_dim, dec_hidden_dim, dec_dropout, attn)

model = seq2seq(enc, dec, device).to(device)

In [23]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

seq2seq(
  (encoder): EncoderBiGRU(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear): Linear(in_features=768, out_features=768, bias=True)
    (gru): GRU(768, 2050, num_layers=2, dropout=0.5, bidirectional=True)
    (out): Linear(in_features=4100, out_features=2050, bias=True)
  )
  (decoder): DecoderGRU(
    (attention): Attention(
      (attn): Linear(in_features=6150, out_features=2050, bias=True)
      (v): Linear(in_features=2050, out_features=1, bias=False)
    )
    (fc): Linear(in_features=768, out_features=768, bias=True)
    (gru): GRU(4868, 2050)
    (out): Linear(in_features=6918, out_features=768, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [17]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [18]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0
  for i, batch in enumerate(iterator):
    src = batch[0]
    trg = batch[1]

    optimizer.zero_grad()

    output = model(src, trg)

    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    optimizer.step()

    epoch_loss += loss.item()

    print('done 1 batch')

  return epoch_loss / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
  model.eval()
  epoch_loss = 0

  with torch.no_grad():
    for i, batch in enumerate(iterator):
      src = text
      trg = target

      output = model(src, trg, 0)
      output_dim = output.shape[-1]
      output = output[1:].view(-1, output_dim)
      trg = trg[1:].view(-1)

      loss = criterion(output, trg)

      epoch_loss += loss.item()

  return epoch_loss / len(iterator)


In [24]:
n_epochs = 1
clip = 1

best_valid_loss = float('inf')

for epoch in range(n_epochs):
  train_loss = train(model, subtrain_iterator, optimizer, criterion, clip)
  valid_loss = evaluate(model, subval_iterator, criterion) 

  if valid_loss < best_valid_loss:
    best_valid_loss = best_valid_losstorch.save(model.state_dict(), 'model.pt')

1
2
input torch.Size([942, 4, 768]) encoder1
output torch.Size([942, 4, 768]) encoder2
output torch.Size([942, 4, 4100]) hidden torch.Size([4, 2050]) encoder3
3
hidden torch.Size([4, 2050]) decoder0
input torch.Size([1, 4, 768]) decoder1
embedded torch.Size([1, 4, 768]) decoder2
batch_size 4 attention1
src_len 942 attention2
hidden torch.Size([4, 942, 2050]) attention3
encoder_outputs torch.Size([4, 942, 4100]) attention4
energy torch.Size([4, 942, 2050]) attention5
attention torch.Size([4, 942]) attention6
attn torch.Size([4, 1, 942]) decoder3
end_out torch.Size([4, 942, 4100]) decoder4
weighted torch.Size([1, 4, 4100]) decoder5
output torch.Size([4, 2050]) weighted torch.Size([4, 4100]) embedded torch.Size([4, 768]) decoder6
pred torch.Size([4, 768]) hidden torch.Size([1, 4, 2050]) decoder7
output torch.Size([4, 768]) hidden torch.Size([1, 4, 2050]) s2s1
outputs torch.Size([37, 4, 768]) s2s2
top1 torch.Size([1, 768]) s2s3
input torch.Size([1, 768]) s2s4
hidden torch.Size([1, 4, 2050]

RuntimeError: ignored

In [ ]:
model.load_state_dict(torch.load('model.pt'))

test_loss = evaluate(model, test_iterator, criterion)
print(test_loss)

In [43]:
x = torch.rand((4,768))
x.argmax(0).shape

torch.Size([768])